<a href="https://colab.research.google.com/github/IamArmanNikkhah/MinMax-TicTacToe/blob/main/TicTacToe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random, time, sys

In [2]:
class Board:
    def __init__(self):
        self.board = [[None, None, None],
                      [None, None, None],
                      [None, None, None]]

    def make_move(self, move, player):
        if move not in self.get_available_moves() and player != None:
            raise Exception('Invalid move')

        self.board[move[0]][move[1]] = player

    def get_available_moves(self):
        return [(i, j) for i in range(3) for j in range(3) if self.board[i][j] is None]

    def is_game_over(self): 
        return len(self.get_available_moves()) == 0 or self.get_winner() != None
    
    def is_winner(self, player):
        if player is None:
            return False

        for i in range(3):
            if self.board[i][0] == player and self.board[i][1] == player and self.board[i][2] == player:
                return True
            if self.board[0][i] == player and self.board[1][i] == player and self.board[2][i] == player:
                return True

        if self.board[0][0] == player and self.board[1][1] == player and self.board[2][2] == player:
            return True
        if self.board[0][2] == player and self.board[1][1] == player and self.board[2][0] == player:
            return True

        return False

    def get_winner(self):
        for player in ['X', 'O']:
            if self.is_winner(player):
                return player

        return None

    def show_board(self):
        for i in range(3):
            for j in range(3):
                element = self.board[i][j]
                if element == None:
                  element = ' '

                print("|", element, end=" ")
            print("|")
        print("--------------------------")
    
    def __str__(self):
        return '\n'.join([' '.join([str(self.board[i][j]) for j in range(3)]) for i in range(3)])


In [3]:
class Game:
    def __init__(self):
        self.board = Board()
        self.player = random.choice(['X', 'O'])
        self.computer = 'X' if self.player == 'O' else 'O'
        self.minmax = MinMax(self.board, self.computer)

    def is_game_over(self): 
        return len(self.board.get_available_moves()) == 0 or self.board.get_winner() != None
    
    def start(self):
        print('You are \'{}\'. Computer is \'{}\'.'.format(self.player, self.computer))
        print('Rule : Computer starts first.\n')
        self.board.show_board()

        while not self.is_game_over():
            self.computer_turn()
            if self.is_game_over():
              break
            self.player_turn()

        if self.board.is_winner(self.player):
            print('You win!')
        elif self.board.is_winner(self.computer):
            print('Computer wins!')
        else:
            print('Draw!')

    def player_turn(self):
        while True:
            try:
                move = input('Enter move (row, column): ')
                move = move.split(',')
                move = (int(move[0]), int(move[1]))
                self.board.make_move(move, self.player)
                break
            except Exception as e:
                print(e)

        self.board.show_board()

    def computer_turn(self):
        print('Computer is thinking...')
        time.sleep(1)
        tic = time.perf_counter()
        self.minmax.find_best_move()
        self.board.make_move(self.minmax.get_best_move(), self.computer)
        toc = time.perf_counter()
        print('Computer move: {}'.format(self.minmax.get_best_move()))
        print(f"Thinking Time :  {toc - tic:0.4f} seconds")
        self.minmax.reset()
        self.board.show_board()

In [6]:
class MinMax:
    def __init__(self, board, player):
        self.board = board
        self.player = player
        self.opponent = 'X' if player == 'O' else 'O'
        self.best_move = None
        self.best_score = float('-inf')

    def get_best_move(self):
        return self.best_move

    def get_best_score(self):
        return self.best_score

    def reset(self):
      self.best_move = None
      self.best_score = float('-inf')
    
    def heuristic(self, Board):
      board = Board.board
      def winning_possibilities(player):
        counter_winning = 0
        for i in range(3):
            counter_player_row , counter_player_col , counter_player_diag   = 0, 0, 0
            counter_empty_row  , counter_empty_col  , counter_empty_diag    = 0, 0, 0
            if board[i][i] == player:
              counter_player_diag += 1
            if board[i][i] == None:
              counter_empty_diag += 1
            for j in range(3):
              if board[i][j] == player:
                counter_player_row += 1
              if board[i][j] == None:
                counter_empty_row += 1
              if board[j][i] == player:
                counter_player_col += 1
              if board[j][i] == None:
                counter_empty_col += 1
            
            if counter_player_row == 2 and counter_empty_row == 1:
              counter_winning += 1 
            if counter_player_col == 2 and counter_empty_col == 1:
              counter_winning += 1 
        
        if counter_player_diag == 2 and counter_empty_diag == 1:
          counter_winning += 1 

        return counter_winning

      player_wining_possibilities =  winning_possibilities(self.opponent) #
      total_wining_possibilities  =  player_wining_possibilities + winning_possibilities(self.opponent) + sys.float_info.epsilon
      return 2*(player_wining_possibilities/total_wining_possibilities) - 1
        
    
    def minmax(self, board, player, depth, alpha, beta):
        if board.is_game_over():
            if board.is_winner(self.player):
                return 1
            elif board.is_winner(self.opponent):
                return -1
            else:
                return 0

        if depth == 0:
            if player == self.player: return -1*self.heuristic(board)
            else                    : return self.heuristic(board)

        for move in board.get_available_moves():
            board.make_move(move, player)
            score = self.minmax(board, self.opponent if player == self.player else self.player, depth - 1, alpha, beta)
            board.make_move(move, None)

            if player == self.player:
                if score > alpha:
                    alpha = score
                if alpha >= beta:
                    return beta
            else:
                if score < beta:
                    beta = score
                if beta <= alpha:
                    return alpha

        if player == self.player:
            return alpha
        else:
            return beta

    def find_best_move(self):
        for move in self.board.get_available_moves():
            self.board.make_move(move, self.player)
            score = self.minmax(self.board, self.opponent, 3, float('-inf'), float('inf')) 
            self.board.make_move(move, None)

            if score > self.best_score:
                self.best_score = score
                self.best_move = move


In [ ]:
game = Game()
game.start()